In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.linear_model import LogisticRegression,Lasso,Ridge,LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

check missing ratio

In [ ]:
data.isnull().sum()/len(data)*100

Feature 'company' missing ratio is too high,we remove the company feature

In [ ]:
data.drop('company',axis=1,inplace=True)

In [ ]:
data.is_canceled.value_counts()/len(data['is_canceled'])

In [ ]:
plt.pie(data['is_canceled'].value_counts(),autopct='%.2f%%',explode=[0,0.08],shadow=True,labels=['No Canceled','Canceled'])

Feature 'country' is object type,missing ratio is not too much,hence,we use mode() to fill null value

In [ ]:
data['country'].fillna(data['country'].mode()[0],inplace=True)

In [ ]:
data['agent'].value_counts()

Feature 'agent' is numeric,missing ratio is low,we fill na with median

In [ ]:
data['agent'].fillna(data['agent'].notnull().median(),inplace=True)

Feature 'children' is numeric,missing value is very very few,we fill missing value with mean()

In [ ]:
data['children'].fillna(data['children'].notnull().mean(),inplace=True)

In [ ]:
data['hotel'].unique()

In [ ]:
obj_columns=data.select_dtypes(include=['object'])
columns=obj_columns.columns

In [ ]:
columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
for i in range(0,len(columns)):
    data[columns[i]]=le.fit_transform(data[columns[i]])

In [ ]:
data.head()

In [ ]:
data.hotel.value_counts()

In [ ]:
cor=data.corr()
plt.figure(figsize=(24,12))
sns.heatmap(cor,cmap='YlGnBu',vmin=0,vmax=1,annot=True)

In [ ]:
help(sns.heatmap)

In [ ]:
regressions={'Logistic Regression':LogisticRegression(solver='lbfgs'),
           'Linear Regression':LinearRegression(),
           'Ridge Regression':Ridge(alpha=1.0),
           'Lasso Regression':Lasso(alpha=0.1),
           'Random Forest Regression':RandomForestRegressor(n_estimators = 100, random_state = 42),
           'Adaboost Regression':AdaBoostRegressor(n_estimators = 100, random_state = 42)}

In [ ]:
X=data.drop('previous_cancellations',axis=1)
y=data['previous_cancellations']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=111)

In [ ]:
for regression in regressions:
    reg=regressions[regression].fit(X_train,y_train)
    y_pred=reg.predict(X_test)
    print('---'*10,regression,'---'*10)
    print('Mean Absolute Error:',metrics.mean_absolute_error(y_test,y_pred))
    print('Mean Squared Error:',metrics.mean_squared_error(y_test,y_pred))
    print('Root Mean Squared Error:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
    print('r2_score:',r2_score(y_test,y_pred))
    print('---'*25)

> as above,Random Forest Regression is the best model,then we use it to fit the dataset

In [ ]:
rf=RandomForestRegressor(max_features='auto',n_jobs=-1,oob_score=True,random_state=111)
params={'n_estimators':[50,100,150]
       }
gs=GridSearchCV(rf,param_grid=params,cv=5,verbose=0)

In [ ]:
gs.fit(X_train,y_train)

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_